In [1]:
# Imports
from bs4 import BeautifulSoup
from datetime import datetime
from matplotlib.ticker import FuncFormatter
import argparse
import csv
import errno
import json
import locale
import matplotlib.pyplot as plt
import operator
import os
import pandas as pd
import re

os.chdir("..")

import rotinas as rtn

In [2]:
"""Lê dados de parlamentares de arquivos CSV e
gera gráficos, texto e páginas com o conteúdo
"""

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

parser = argparse.ArgumentParser(
    description='Coleta dados de gastos de Senadores brasileiros.')

parser.add_argument('-G', '--nograph', dest='nograph', action='store_true',
                    help='Não gera gráficos')

parser.add_argument('-P', '--nopage', dest='nopage', action='store_true',
                    help='Não gera a página html')

parser.add_argument('-l', '--legislatura', dest='legislatura', type=int, default=55,
                    help='Legislatura de coleta de dados, default: legislatura atual')

#args = parser.parse_args()

_StoreAction(option_strings=['-l', '--legislatura'], dest='legislatura', nargs=None, const=None, default=55, type=<class 'int'>, choices=None, help='Legislatura de coleta de dados, default: legislatura atual', metavar=None)

In [4]:
args = parser.parse_args("-l 55 -P -G".split())
args

Namespace(legislatura=55, nograph=True, nopage=True)

In [5]:
legislaturaLevantamento = args.legislatura

# Lê legislatura e Lista de anos de mandato para contabilização
with open(f'csv/{legislaturaLevantamento}_anos.csv', newline='') as arquivoAnos:
    anosReader = csv.reader(arquivoAnos)
    for row in anosReader:
        # Ignora o header (se houver)
        if rtn.maiorQue(row[0]) and rtn.maiorQue(row[1]) and rtn.maiorQue(row[2]):
            legislaturaAtual = int(row[0])
            anos = list(range(int(row[1]), int(row[2]) + 1))
            # Quarto campo está no formato aaaa-mm-dd hh:mm:ss.dcmm
            # Primeiro separa data de hora
            dataColeta, horaColeta = row[3].split(' ')
            # Muda data coleta de aaaa-mm-dd para dd/mm/aaaa
            dataColeta = dataColeta.split('-')
            dataColeta = dataColeta[2] + '/' + \
                dataColeta[1] + '/' + dataColeta[0]
            # Descarta os décimos de segundo
            horaColeta = horaColeta.split('.')[0]
            break
arquivoAnos.close()

anoAtual = datetime.today().year
anos = [ano for ano in anos if ano <= anoAtual]

# Lê créditos das fotos
with open('csv/creditos.csv', newline='') as creditos:
    creditosReader = csv.reader(creditos)
    header = next(creditosReader)
    listaCredito = {}
    for row in creditosReader:
        listaCredito[int(row[0].split('.')[0].replace('senador', ''))] = row[1]
creditos.close()

# Lê DataFrames
dadosSenado = pd.read_csv(f'csv/{legislaturaLevantamento}_senado.csv', encoding='utf-8', index_col=0)
top = pd.read_csv(f'csv/{legislaturaLevantamento}_top.csv', encoding='utf-8')
gastoPartidos = pd.read_csv(f'csv/{legislaturaLevantamento}_gastoPartidos.csv',
                            encoding='utf-8', index_col=0)
gastoEstados = pd.read_csv(f'csv/{legislaturaLevantamento}_gastoEstados.csv',
                           encoding='utf-8', index_col=0)
#sexo = pd.read_csv('csv/sexo.csv', encoding='utf-8')
sexo = dadosSenado.rename(columns={'Participacao': '(Sexo, Situação)'}).groupby(
    ['sexo', 'status'])['(Sexo, Situação)'].count()
sexoT = pd.read_csv(f'csv/{legislaturaLevantamento}_sexoT.csv', encoding='utf-8', index_col=0)

# Lê arquivo json
with open(f'json/{legislaturaLevantamento}_gastosSenadores.json', 'r', encoding='utf-8') as entrada:
    gastosSenadores = json.load(entrada)
entrada.close()

# Calcula dados importantes
totalSenadores = len(dadosSenado)
totalHomens = len(dadosSenado[dadosSenado.sexo == 'Masculino'])
totalMulheres = len(dadosSenado[dadosSenado.sexo == 'Feminino'])
totalExercicio = len(dadosSenado[dadosSenado.status == 'Exercicio'])
totalMulheresExercicio = dadosSenado.query(
    'sexo == "Feminino" and status == "Exercicio"').count()[0]
totalForaExercicio = len(dadosSenado[dadosSenado.status == 'ForaExercicio'])
totalGasto = dadosSenado['gastos'].sum()

# Não contabiliza parlamentares que ainda não efetuaram gastos no cálculo de médias
gastoMedioSenadores = dadosSenado.query('gastos != 0')['gastos'].mean()
mediaGastosHomensExercicio = dadosSenado.query(
    'gastos != 0 and sexo == "Masculino" and status == "Exercicio"')['gastos'].mean()
mediaGastosMulheresExercicio = dadosSenado.query(
    'gastos !=0 and sexo == "Feminino" and status == "Exercicio"')['gastos'].mean()


# Imprime algumas informações do senado, pelos dados coletados
print(f'Há no senado {totalSenadores:d} senadores, distribuidos entre {totalHomens:d} homens e {totalMulheres:d} mulheres')
print('As mulheres representam ' + locale.format('%.2f',
                                                 totalMulheres / totalSenadores * 100) + '% do total')
print('Há {:d} senadores em exercício, destes {:d} são mulheres'.format(
    totalExercicio, totalMulheresExercicio))
print('As mulheres representam ' + locale.format('%.2f',
                                                 totalMulheresExercicio / totalExercicio * 100) + '% deste total')
print('O gasto médio de senadores homens em exercício foi de ' +
      rtn.reais(mediaGastosHomensExercicio))
print('O gasto médio de senadores mulheres em exercício foi de ' +
      rtn.reais(mediaGastosMulheresExercicio))
print('O gasto médio dos senadores, em exercício e fora de exercício, foi de ' +
      rtn.reais(gastoMedioSenadores))
print('O montante de despesas parlamentares em {:d} anos foi de {}, com media anual de {}\n'.format(
    len(anos), rtn.reais(totalGasto), rtn.reais(totalGasto / len(anos))))

Há no senado 128 senadores, distribuidos entre 113 homens e 15 mulheres
As mulheres representam 11,72% do total
Há 81 senadores em exercício, destes 13 são mulheres
As mulheres representam 16,05% deste total
O gasto médio de senadores homens em exercício foi de R$ 1.174.243,20
O gasto médio de senadores mulheres em exercício foi de R$ 1.106.002,31
O gasto médio dos senadores, em exercício e fora de exercício, foi de R$ 948.375,13
O montante de despesas parlamentares em 4 anos foi de R$ 100.527.764,00, com media anual de R$ 25.131.941,00



In [6]:
# Totaliza os gastos do senado por tipo de gasto
gastosSenado = {}
for senador in gastosSenadores:
    for gastos in senador['gastos']:
        for caput in gastos['lista']:
            if not caput in gastosSenado:
                gastosSenado[caput] = gastos['lista'][caput]
            else:
                gastosSenado[caput] += gastos['lista'][caput]

gastosSenado = {caput: round(gastosSenado[caput], 2) for caput in gastosSenado}

print("Gastos do senado por tema:")
totalizacaoGastosSenado = 0.0
for caput in gastosSenado:
    totalizacaoGastosSenado += gastosSenado[caput]
    print('{}: {}'.format(caput, rtn.reais(round(gastosSenado[caput], 2))))

print('Total de gastos: {}'.format(rtn.reais(round(totalizacaoGastosSenado, 2))))

Gastos do senado por tema:
Aluguel de imóveis para escritório político: R$ 16.294.875,33
Aquisição de material de consumo: R$ 2.993.966,93
Locomoção, hospedagem, alimentação, combustíveis e: R$ 19.074.567,03
Divulgação da atividade parlamentar: R$ 11.617.313,02
Passagens aéreas, aquáticas e terrestres nacionais: R$ 21.347.632,71
Diárias: R$ 1.999.783,70
Combustíveis: R$ 965.177,49
Outros materiais: R$ 1.261.923,35
Correios: R$ 10.220.199,77
Contratação de serviços de apoio ao parlamentar: R$ 11.611.313,82
Ressarcimento de Passagens: R$ 1.015.430,57
Serviços de Segurança Privada: R$ 1.493.563,30
Total de gastos: R$ 99.895.747,02
